# 보안, 금융, 위험 관리, 법률, 재해 복구, IT 거버넌스

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -U langchain-community
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00


In [ ]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 5.6 MB/s eta 0:00:00


In [ ]:
!pip install -U bitsandbytes transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install peft

In [ ]:
import re
import json
import torch
import pandas as pd

from tqdm import tqdm
from datasets import Dataset, load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from typing import List, Dict
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, pipeline, BitsAndBytesConfig
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

import warnings
warnings.filterwarnings("ignore")

# PDF Load & Split Chunk

In [152]:
%%time
loader = PyPDFLoader("/content/drive/MyDrive/1데이콘/2025금융AIChallenge금융AI모델경쟁/dataset/2025년도 금융감독개론.pdf")
all_chunks = loader.load_and_split()

text_spliter = RecursiveCharacterTextSplitter(
    chunk_size = 300,
    chunk_overlap = 50,
    separators = ["\n\n", "\n", " ", ""]
)
documents = loader.load()
all_chunks_custom = text_spliter.split_documents(documents)
# all_chunks = loader.load_and_split(text_spliter = text_spliter) 위에꺼랑 같은 결과

print("청크의 수 :", len(all_chunks))
print("청크의 수 :", len(all_chunks_custom))

청크의 수 : 720
청크의 수 : 3200
CPU times: user 1min, sys: 79.7 ms, total: 1min
Wall time: 1min


In [ ]:
# 처음 불필요한 부분 확인
all_chunks[5].page_content

'제1장 금융감독·검사 기초 ◆ 3\n제1장\n금융감독·검사 기초\n제1절 금융감독의 개념과 필요성\n1. 금융감독의 개념\n금융감독의 개념을 명확하게 정의하기는 쉽지 않다. 1980년대 초반부터 BIS 등 국제 금융 \n감독기구에서 금융감독에 대한 이론적 접근이 시도되었지만 아직 만족할 만한 성과가 나오지  \n않고 있다. 다만, 1997.6월 영란은행(Bank of England)이 주최한 제4회 세계 중앙은행  \n총재단 심포지엄에서 “금융감독과 금융규제는 본질적으로 적절한 유인 구조를 창출함으로써  \n금융회사와 금융시장 참가자의 행태를 바꾸는 것에 관한 것”이라는 정의를 발표한 바 있어  \n참고할 만하다.\n그러나 이러한 정의는 금융감독이 기본적으로 따라야 할 속성을 포괄적으로 제시하고는  \n있지만, 누가, 무엇을, 어떻게 감독하여야 하는가에 대한 명쾌한 답이 되지는 못하고 있다.  \n이런 배경 때문에 나라마다 금융감독의 주체, 대상 및 그 방법과 절차 등이 차이를 보이고  \n있다. 그렇지만 금융감독기구의 지향점에 대해서는 금융시스템의 안전성, 금융회사의 건전성과  \n안정성 유지 그리고 금융소비자 보호를 목적으로 해야 한다는 데에 세계 각국이 공감대를  \n형성하고 있다.1)\n1) David T. Llewellyn, “The Economic Rationale for Financial Regulation,” 1999.'

In [ ]:
# 마지막 불필요한 부분 확인
all_chunks[-20].page_content

'제8장 금융소비자보호 ◆ 707\n5. 사회적금융 활성화\n사회적금융이란 단순한 보조 · 기부행위가 아니라 사회적경제기업(사회적기업, 협동조합, \n자활기업, 소셜벤처기업 등)에 회수를 전제로 자금을 공급함으로써 사회적 가치의 실현과 \n재무적 이익을 동시에 추구하는 금융을 말한다. 2018.2월 정부는 사회적가치를 추구하는 \n사회적경제기업 육성을 위한 ‘사회적금융 활성화 방안(사회적경제 활성화 지원을 위한 범정 \n부대책)’을 발표하였다. 이에 금융감독원도 은행권과 협력체계를 구축하여 민간분야의 자율 \n적인 사회적금융 공급을 유도하는 한편, 은행권 사회적금융 모범규준(은행연합회) 제정,  \n사회적금융 전용 홈페이지 구축, 우수 금융회사 표창 수여 등을 통해 각종 제도적 기반을 \n정비하였다.\n은행권의 2024년 9월말 기준 사회적경제기업에 대한 대출잔액은 1조 7,033억원으로 \n2023년말(1조 6,616억원) 대비 2.5%(417억원) 증가하였으며 대출 이외의 지원(기부 · 후원, \n제품구매 등) 실적은 2024년 3분기 기준 84.0억원이었다.'

In [ ]:
# 4페이지까지 제외
all_chunks = all_chunks[5:]
print("줄어든 청크 수 :", len(all_chunks))

줄어든 청크 수 : 715


In [ ]:
# 마지막 19페이지 제외
all_chunks = all_chunks[:-20]
print("줄어든 청크 수 :", len(all_chunks))

줄어든 청크 수 : 695


# QA 데이터로 변환

In [ ]:
# model_name = "beomi/KoAlpaca-Polyglot-5.8B"

# qa_gen = pipeline("text-generation", model = model_name, device = 0)

# qa_dataset = []

# for i, doc in enumerate(docs):

#     prompt = f"""
#     아래 문단을 바탕으로 객관식 질문 1개와 주관식 질문 1개를 만들어줘.
#     객관식은 4지선다로 하고 정답 표시 포함.

#     문단 :
#     {doc.page_content}
#     """

#     result = qa_gen(prompt, max_new_tokens = 128)[0]["generated_text"]

#     qa_dataset.append({
#         "context" : doc.page_content,
#         "qa" : result
#     })

In [ ]:
model_name = "skt/A.X-4.0-Light"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.padding_side = 'left'

qa_gen = pipeline("text-generation",
                  model = model_name,
                  tokenizer = tokenizer,
                  device=0)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.60G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/130 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
contexts = [doc.page_content for doc in all_chunks]

prompt = [
    f"""
    아래 문단을 바탕으로 객관식 질문 1개와 주관식 질문 1개를 만들어줘.
    객관식은 4지선다로 하고 정답 표시 포함.

    문단 :
    {context}
    """
    for context in contexts
]

In [ ]:
results = qa_gen(prompt, max_new_tokens=128)

# context가 문자열 리스트
qa_dataset = [
    {"context": contexts[i], "qa": res[0]["generated_text"]}
    for i, res in enumerate(results)
]

# context가 문서 객체 리스트
# qa_dataset = [
#     {'context': doc.page_content, 'qa': res[0]['generated_text']}
#     for doc, res in zip(contexts, results)
# ]

In [ ]:
print(len(qa_dataset))

695


In [ ]:
# qa_dataset 저장
with open("/content/drive/MyDrive/1데이콘/2025 금융 AI Challenge : 금융 AI 모델 경쟁/dataset/qa_dataset.json", "w", encoding = "utf-8") as f:
    for item in qa_dataset:
        json.dump({
            "instruction" : "다음 문단에 대한 질문에 답하시오.",
            "input" : item["context"],
            "output" : item["qa"]
        }, f, ensure_ascii = False)
        f.write("\n")

# Model Load & Train, Save

In [ ]:
# model_name = "skt/A.X-4.0-Light"

In [ ]:
for name, module in model.named_modules():
    if "attn" in name.lower() or "attention" in name.lower():
        print(name)

model.layers.0.self_attn
model.layers.0.self_attn.q_proj
model.layers.0.self_attn.k_proj
model.layers.0.self_attn.v_proj
model.layers.0.self_attn.o_proj
model.layers.0.post_attention_layernorm
model.layers.1.self_attn
model.layers.1.self_attn.q_proj
model.layers.1.self_attn.k_proj
model.layers.1.self_attn.v_proj
model.layers.1.self_attn.o_proj
model.layers.1.post_attention_layernorm
model.layers.2.self_attn
model.layers.2.self_attn.q_proj
model.layers.2.self_attn.k_proj
model.layers.2.self_attn.v_proj
model.layers.2.self_attn.o_proj
model.layers.2.post_attention_layernorm
model.layers.3.self_attn
model.layers.3.self_attn.q_proj
model.layers.3.self_attn.k_proj
model.layers.3.self_attn.v_proj
model.layers.3.self_attn.o_proj
model.layers.3.post_attention_layernorm
model.layers.4.self_attn
model.layers.4.self_attn.q_proj
model.layers.4.self_attn.k_proj
model.layers.4.self_attn.v_proj
model.layers.4.self_attn.o_proj
model.layers.4.post_attention_layernorm
model.layers.5.self_attn
model.laye

In [128]:
# 데이터 로드
dataset = load_dataset("json", data_files = "/content/drive/MyDrive/1데이콘/2025금융AIChallenge금융AI모델경쟁/dataset/qa_dataset.json")

# 양자화
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_use_double_quant = True,
    bnb_4bit_compute_dtype = torch.bfloat16
)

# 모델, 토크나이저 로드
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map = "auto",
    quantization_config = bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 그래디언트 체크포인팅 활성화(메모리 절약)
model.gradient_checkpointing_enable()

# 모델을 4비트 훈련에 적합하게 조성
model = prepare_model_for_kbit_training(model)

# LoRA 설정
lora_config = LoraConfig(
    lora_alpha = 32,
    lora_dropout = 0.05,
    r = 16,
    target_modules = ['q_proj',
                      'k_proj',
                      'v_proj',
                      'o_proj'
    ]
)

# 모델에 LoRA 적용
model = get_peft_model(model, lora_config)

# 훈련 가능한 파라미터 확인
model.print_trainable_parameters()

Generating train split: 0 examples [00:00, ? examples/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

trainable params: 10,092,544 || all params: 7,269,717,504 || trainable%: 0.1388


In [ ]:
# dataset = dataset["train"]
tokenizer.pad_token = tokenizer.eos_token  # pad_token 설정 (필수)

# 3. 프롬프트 포맷 정의
def format_prompt(ex):
    if ex["input"]:
        return f"### Instruction:\n{ex['instruction']}\n\n### Input:\n{ex['input']}\n\n### Response:\n{ex['output']}"
    else:
        return f"### Instruction:\n{ex['instruction']}\n\n### Response:\n{ex['output']}"

# 4. 토크나이징 함수
def preprocess_function(ex):
    prompt = format_prompt(ex)
    tokenized = tokenizer(prompt, truncation=True, padding="max_length", max_length=1024)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

# 5. 전처리 적용
tokenized_dataset = dataset.map(
    preprocess_function,
    batched=False,
    remove_columns=dataset.column_names
)

In [113]:
# training argument 설정
training_args = TrainingArguments(
    output_dir = "/content/drive/MyDrive/1데이콘/2025 금융 AI Challenge : 금융 AI 모델 경쟁/dataset/finetunning_model",
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 8,
    learning_rate = 2e-5,
    num_train_epochs = 3,
    logging_steps = 50,
    save_steps = 500,
    # fp16 = True,
    bf16 = True, # gpu 지원
    remove_unused_columns=False
)

# 학습
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_dataset,
    tokenizer = tokenizer
)

trainer.train()
trainer.save_model()

Step,Training Loss
50,1.158900
100,1.059500


# Test data Load & 후처리

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/1데이콘/2025 금융 AI Challenge : 금융 AI 모델 경쟁/dataset/test.csv')

In [ ]:
# 객관식 여부 판단 함수
def is_multiple_choice(question_text):
    """
    객관식 여부를 판단: 2개 이상의 숫자 선택지가 줄 단위로 존재할 경우 객관식으로 간주
    """
    lines = question_text.strip().split("\n")
    option_count = sum(bool(re.match(r"^\s*[1-9][0-9]?\s", line)) for line in lines)
    return option_count >= 2


# 질문과 선택지 분리 함수
def extract_question_and_choices(full_text):
    """
    전체 질문 문자열에서 질문 본문과 선택지 리스트를 분리
    """
    lines = full_text.strip().split("\n")
    q_lines = []
    options = []

    for line in lines:
        if re.match(r"^\s*[1-9][0-9]?\s", line):
            options.append(line.strip())
        else:
            q_lines.append(line.strip())

    question = " ".join(q_lines)
    return question, options

In [ ]:
# 프롬프트 생성기
def make_prompt_auto(text):
    if is_multiple_choice(text):
        question, options = extract_question_and_choices(text)
        prompt = (
                "당신은 금융보안 전문가입니다.\n"
                "아래 질문에 대해 적절한 **정답 선택지 번호만 출력**하세요.\n\n"
                f"질문: {question}\n"
                "선택지:\n"
                f"{chr(10).join(options)}\n\n"
                "답변:"
                )
    else:
        prompt = (
                "당신은 금융보안 전문가입니다.\n"
                "아래 주관식 질문에 대해 정확하고 간략한 설명을 작성하세요.\n\n"
                f"질문: {text}\n\n"
                "답변:"
                )
    return prompt

In [ ]:
# 후처리 함수
def extract_answer_only(generated_text: str, original_question: str) -> str:
    """
    - "답변:" 이후 텍스트만 추출
    - 객관식 문제면: 정답 숫자만 추출 (실패 시 전체 텍스트 또는 기본값 반환)
    - 주관식 문제면: 전체 텍스트 그대로 반환
    - 공백 또는 빈 응답 방지: 최소 "미응답" 반환
    """
    # "답변:" 기준으로 텍스트 분리
    if "답변:" in generated_text:
        text = generated_text.split("답변:")[-1].strip()
    else:
        text = generated_text.strip()

    # 공백 또는 빈 문자열일 경우 기본값 지정
    if not text:
        return "미응답"

    # 객관식 여부 판단
    is_mc = is_multiple_choice(original_question)

    if is_mc:
        # 숫자만 추출
        match = re.match(r"\D*([1-9][0-9]?)", text)
        if match:
            return match.group(1)
        else:
            # 숫자 추출 실패 시 "0" 반환
            return "0"
    else:
        return text

# Fine-tuning Model Load

In [135]:
from peft import PeftModel

adapter_path = "/content/drive/MyDrive/1데이콘/2025금융AIChallenge금융AI모델경쟁/dataset/finetunning_model/checkpoint-132"

fine_model = PeftModel.from_pretrained(model, adapter_path)
model = fine_model.merge_and_unload()

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cuda:0


# Inference

In [136]:
preds = []

for q in tqdm(test['Question'], desc="Inference"):
    prompt = make_prompt_auto(q)
    output = pipe(prompt, max_new_tokens=128, temperature=0.2, top_p=0.9)
    pred_answer = extract_answer_only(output[0]["generated_text"], original_question=q)
    preds.append(pred_answer)

Inference: 100%|██████████| 515/515 [07:44<00:00,  1.11it/s]


# Submission

In [151]:
sample_submission = pd.read_csv('/content/drive/MyDrive/1데이콘/2025금융AIChallenge금융AI모델경쟁/dataset/sample_submission.csv')
sample_submission['Answer'] = preds
sample_submission.to_csv('/content/drive/MyDrive/1데이콘/2025금융AIChallenge금융AI모델경쟁/submission_final.csv', index=False, encoding='utf-8-sig')

from google.colab import files
file_path = "/content/drive/MyDrive/1데이콘/2025금융AIChallenge금융AI모델경쟁/submission_final.csv"
files.download(file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>